<a href="https://colab.research.google.com/github/IS5882/UU-TAX/blob/main/UU_Tax_Subtask_2_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UU TaX at SemEval 2022: *TASK3: PRETENS - SUBTASK2*




Cloning the git repository, to get data for Task3:subtask-2


# Imports

In [1]:
!pip install tensorflow_text 

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from matplotlib import pyplot

In [3]:
!pip3 install tensorflow_text>=2.0.0rc0


In [4]:
##### Data Reader -- English/French/Italian
!git clone https://github.com/shammur/SemEval2022Task3.git
%cd ./SemEval2022Task3/
#data/train/train_subtask-2/
!ls

Cloning into 'SemEval2022Task3'...
remote: Enumerating objects: 391, done.
remote: Counting objects: 100% (391/391), done.
remote: Compressing objects: 100% (325/325), done.
remote: Total 391 (delta 182), reused 155 (delta 53), pack-reused 0
Receiving objects: 100% (391/391), 2.05 MiB | 3.37 MiB/s, done.
Resolving deltas: 100% (182/182), done.
/content/SemEval2022Task3
baseline	  README.md
data		  SemEval_Task3_Baseline_subtask1.ipynb
LICENSE		  SemEval_Task3_Baseline_subtask2.ipynb
pretens_logo.png  Task-timeline


Setting up the environment for the task

In [5]:
import os, sys, time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing
from sklearn.svm import SVR
from scipy import stats
from collections import defaultdict
import numpy as np
import pandas as pd
import statistics

from baseline.utils import get_train_dev, compute_metrics_task2, process_eval_jsons_task2

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)



# Download USE


In [6]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text


model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3") #large



#  Regression

## Models

### USE

In [16]:
def use(train,test, classifier='svm'):
  eval_jsons=[]

  def getUse(df):
    return model([df['Sentence']])[0].numpy()


  #Get Features 
  train['Sentence_emb']=train.apply(getUse,axis=1)
  train_x_feat=train['Sentence_emb']


  test['Sentence_emb']=test.apply(getUse,axis=1)
  test_x_feat=test['Sentence_emb']

  #Get text and lables
  train_texts=(train['Sentence'])
  y_train=np.array(train['Score'])
  
  test_texts=(test['Sentence'])
  test_true_y=np.array(test['Scores'])   

  
  if classifier=='lr':
        regresson =LinearRegression().fit(list(train_x_feat), y_train)
  elif classifier=='knr':
        regresson =KNeighborsRegressor().fit(list(train_x_feat), y_train)
  elif classifier=='dt':
        regresson =DecisionTreeRegressor().fit(list(train_x_feat), y_train)
  else:
      regresson =SVR(C=1.0, epsilon=0.2).fit(list(train_x_feat), y_train)
    
  #Testing:
  test_pred_y = regresson.predict(list(test_x_feat))
    
  eval_jsons.append(compute_metrics_task2(test_pred_y,test_true_y))
  return eval_jsons,test_pred_y

### TF_IDF and and send classifier name: svm, knr, lr, dt (For baseline testing)

In [17]:
def tfidf(train,test, classifier='svm', ngram=3):
  eval_jsons=[]

 

  #Get text and lables
  train_texts=(train['Sentence'])
  y_train=np.array(train['Score'])
  
  test_texts=(test['Sentence'])
  test_true_y=np.array(test['Scores'])   

  #Creating ngram features
  tfidf_vectorizer = TfidfVectorizer(encoding='utf-8', lowercase=True, ngram_range=(1, ngram), norm='l1',
                                       use_idf=True, max_df=0.95, min_df=3, max_features=1000) 

  train_x_feat = tfidf_vectorizer.fit_transform(train_texts)


  #Training Linear Model
  if classifier=='lr':
        regresson =LinearRegression().fit(train_x_feat, y_train)
  elif classifier=='knr':
        regresson =KNeighborsRegressor().fit(train_x_feat, y_train)
  elif classifier=='dt':
        regresson =DecisionTreeRegressor().fit(train_x_feat, y_train)
  else:
        regresson =SVR(C=1.0, epsilon=0.2).fit(train_x_feat, y_train)
    
  #Testing:
  test_x_feat = tfidf_vectorizer.transform(test_texts)
  test_pred_y = regresson.predict(test_x_feat)
    
  eval_jsons.append(compute_metrics_task2(test_pred_y,test_true_y))
  return eval_jsons,test_pred_y

# English

## Train

In [18]:
data_url = 'https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/train/train_subtask-2/en/'
folds = [pd.read_csv(os.path.join(data_url, 'En-Subtask2-fold_{}.tsv'.format(i)), sep='\t') \
         for i in range(2)] #reading all folds from GitHub

df_en_train = pd.concat(folds[0:2]) 
df_en_train

,ID,Sentence,Score
0,34,"I like cats, but not sparrows.",4.77
1,256,"I like caviar, an interesting type of veal.",1.83
2,118,"I like turkey, but not oysters.",5.42
3,363,"I like crabs, and chicken too.",5.75
4,307,"I like oaks, and trees too.",4.00
...,...,...,...
257,248,"I like oysters, an interesting type of turkey.",1.83
258,277,"I like ducks, and birds too.",4.83
259,255,"I like seafood, an interesting type of caviar.",1.67
260,375,"I like oysters, and seafood too.",4.45


## Read  test data

In [19]:
#dummy score
import random
scores=[random.uniform(1.0,7.0)]*1009 
len(scores)

1009

In [20]:
import base64
import requests
import pandas as pd
df_en_test = pd.read_csv('https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/test/official_test_set_with_labels/subtask-2/En-Subtask2-scores.tsv', sep='\t')
df_en_test=df_en_test.set_index('ID')
#df_en_test['Score']=scores
df_en_test 

,Construction,Sentence,Scores
ID,,,
en_0,comparatives,I like governors more than farmers.,5.83
en_1,unlike,"Unlike parrots, cats are often mentioned in th...",4.83
en_2,unlike,"Unlike pines, bushes are often mentioned in th...",4.42
en_3,ingeneral,"I like oaks, and bushes in general.",4.22
en_4,ingeneral,"I like beef, and oysters in general.",2.58
...,...,...,...
en_1004,particular,"I like hamsters, and in particular sparrows.",1.67
en_1005,unlike,"Unlike forests, concrete is often mentioned in...",4.33
en_1006,particular,"I like trees, and in particular oaks.",6.33


## Call best model and Save best output English

In [21]:
classifiers=['svm','knr','dt','lr']
max_rho=(float('-inf'),"")

In [23]:
for classifier in classifiers:
  results,scores=use(df_en_train,df_en_test,classifier)
  print('English results on model--USE--->',classifier)
  print(results)

English results on model--USE---> svm
[{'mse': 1.7739946655595118, 'rmse': 1.3319139107162714, 'rho': 0.4780457951397609, 'p-value': 9.664365788469291e-59, 'eval_pred': array([4.83009541, 4.38265254, 4.16041709, ..., 3.05867933, 2.36724705,
       3.96661343]), 'eval_labels': array([5.83, 4.83, 4.42, ..., 6.33, 1.25, 3.17])}]
English results on model--USE---> knr
[{'mse': 2.5861947869177406, 'rmse': 1.608165037214073, 'rho': 0.2352512177404732, 'p-value': 3.734456062999287e-14, 'eval_pred': array([5.422, 4.79 , 4.856, ..., 3.66 , 2.094, 4.828]), 'eval_labels': array([5.83, 4.83, 4.42, ..., 6.33, 1.25, 3.17])}]
English results on model--USE---> dt
[{'mse': 4.319979088206144, 'rmse': 2.07845593848081, 'rho': 0.1177786819629239, 'p-value': 0.00017709440082689181, 'eval_pred': array([3.  , 4.58, 6.  , ..., 1.5 , 2.75, 4.56]), 'eval_labels': array([5.83, 4.83, 4.42, ..., 6.33, 1.25, 3.17])}]
English results on model--USE---> lr
[{'mse': 46243.96544919399, 'rmse': 215.04410117274546, 'rho': 

In [24]:
for ngram in range(2,5):  
  for classifier in classifiers:
    results,scores=tfidf(df_en_train,df_en_test,classifier,ngram)
    print('English results on model--TF-IDF--->',classifier,' --> n grams:  ',ngram)
    print(results)

English results on model--TF-IDF---> svm  --> n grams:   2
[{'mse': 5.035104242701886, 'rmse': 2.243903795331227, 'rho': 0.2465675098369179, 'p-value': 1.93509937182417e-15, 'eval_pred': array([6.13397774, 5.6859524 , 5.61527975, ..., 4.51482026, 0.1411881 ,
       5.67590369]), 'eval_labels': array([5.83, 4.83, 4.42, ..., 6.33, 1.25, 3.17])}]
English results on model--TF-IDF---> knr  --> n grams:   2
[{'mse': 3.665286299306244, 'rmse': 1.914493744911757, 'rho': 0.0767814720478785, 'p-value': 0.014706632414229776, 'eval_pred': array([3.428, 5.274, 4.782, ..., 4.384, 2.298, 4.58 ]), 'eval_labels': array([5.83, 4.83, 4.42, ..., 6.33, 1.25, 3.17])}]
English results on model--TF-IDF---> dt  --> n grams:   2
[{'mse': 4.693415956392467, 'rmse': 2.166429310268966, 'rho': 0.13449971340364106, 'p-value': 1.814723657960269e-05, 'eval_pred': array([3.5 , 5.75, 4.25, ..., 5.67, 2.74, 5.75]), 'eval_labels': array([5.83, 4.83, 4.42, ..., 6.33, 1.25, 3.17])}]
English results on model--TF-IDF---> lr  

call final model

In [25]:
en_results,en_scores=use(df_en_train,df_en_test,'svm')
en_results

[{'eval_labels': array([5.83, 4.83, 4.42, ..., 6.33, 1.25, 3.17]),
  'eval_pred': array([4.83009541, 4.38265254, 4.16041709, ..., 3.05867933, 2.36724705,
         3.96661343]),
  'mse': 1.7739946655595118,
  'p-value': 9.664365788469291e-59,
  'rho': 0.4780457951397609,
  'rmse': 1.3319139107162714}]

In [26]:
df_en_test['Score']=en_scores
del df_en_test['Sentence_emb']
df_en_test

,Construction,Sentence,Scores,Score
ID,,,,
en_0,comparatives,I like governors more than farmers.,5.83,4.830095
en_1,unlike,"Unlike parrots, cats are often mentioned in th...",4.83,4.382653
en_2,unlike,"Unlike pines, bushes are often mentioned in th...",4.42,4.160417
en_3,ingeneral,"I like oaks, and bushes in general.",4.22,4.200822
en_4,ingeneral,"I like beef, and oysters in general.",2.58,4.418871
...,...,...,...,...
en_1004,particular,"I like hamsters, and in particular sparrows.",1.67,3.610609
en_1005,unlike,"Unlike forests, concrete is often mentioned in...",4.33,3.990366
en_1006,particular,"I like trees, and in particular oaks.",6.33,3.058679


In [27]:
from google.colab import files
df_en_test.to_csv('answer_english_with_sent.csv') 
files.download('answer_english_with_sent.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# French

## Train

In [28]:
data_url = 'https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/train/train_subtask-2/fr/'
folds = [pd.read_csv(os.path.join(data_url, 'Fr-Subtask2-fold_{}.tsv'.format(i)), sep='\t') \
         for i in range(2)] #reading all folds from GitHub

df_fr_train = pd.concat(folds[0:2]) 
df_fr_train

,ID,Sentence,Score
0,110,"J'adore le boeuf, mais pas les crabes.",5.08
1,230,"J'adore le boeuf, un type intéressant de saumon.",1.00
2,36,"J'aime les oiseaux, mais pas les moineaux.",5.92
3,128,"J'adore la dinde, mais pas le caviar.",5.92
4,114,"J'adore les produits de la mer, mais pas les h...",6.08
...,...,...,...
257,224,"J'adore le veau, un type intéressant de saumon.",1.08
258,249,"J'adore la dinde, un type intéressant de huîtres.",1.17
259,143,"J'aime les oiseaux, un type intéressant de chien.",1.42
260,88,"J'adore le poulet, mais pas le saumon.",6.08


In [29]:
import base64
import requests
import pandas as pd
df_fr_test = pd.read_csv('https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/test/official_test_set_with_labels/subtask-2/Fr-Subtask2-scores.tsv', sep='\t')
df_fr_test=df_fr_test.set_index('ID')
scores=[random.uniform(1.0,7.0)]*1009 

df_fr_test['Score']=scores
df_fr_test 

,Construction,Sentence,Scores,Score
ID,,,,
fr_0,ingeneral,"J'aime les buissons, et les sapins en général.",3.25,2.794975
fr_1,ingeneral,"J'aime les produits de la mer, et le caviar en...",4.08,2.794975
fr_2,particular,"J'apprécie les conducteurs, et en particulier ...",1.67,2.794975
fr_3,particular,"J'apprécie les politiciens, et en particulier ...",2.42,2.794975
fr_4,ingeneral,"J'aime les huîtres, et le veau en général.",3.25,2.794975
...,...,...,...,...
fr_1004,particular,"J'apprécie les maires, et en particulier les c...",1.50,2.794975
fr_1005,particular,"J'aime les arbres, et en particulier les buiss...",3.83,2.794975
fr_1006,unlike,"Contrairement à le saumon, le poulet est souve...",4.42,2.794975


## Call best model and Save best output French

In [30]:
classifiers=['svm','knr','dt','lr']
max_rho=(float('-inf'),"")

In [31]:
for classifier in classifiers:
  results,scores=use(df_fr_train,df_fr_test,classifier)
  print('French results on model--USE--->',classifier)
  print(results)

French results on model--USE---> svm
[{'mse': 3.3313415574707665, 'rmse': 1.8251963065573977, 'rho': 0.029937021610930405, 'p-value': 0.34212419795352644, 'eval_pred': array([5.36274701, 4.28460411, 3.53859666, ..., 3.86386237, 3.96695503,
       4.16541119]), 'eval_labels': array([3.25, 4.08, 1.67, ..., 4.42, 1.5 , 6.67])}]
French results on model--USE---> knr
[{'mse': 4.354059738354808, 'rmse': 2.0866383822681898, 'rho': 0.10291304029711668, 'p-value': 0.001061674702351423, 'eval_pred': array([5.8  , 4.214, 3.486, ..., 5.2  , 5.15 , 3.658]), 'eval_labels': array([3.25, 4.08, 1.67, ..., 4.42, 1.5 , 6.67])}]
French results on model--USE---> dt
[{'mse': 6.570036372646184, 'rmse': 2.5632082187458325, 'rho': -0.0592669274494624, 'p-value': 0.05984679914293181, 'eval_pred': array([6.33, 2.83, 6.  , ..., 6.  , 5.58, 4.33]), 'eval_labels': array([3.25, 4.08, 1.67, ..., 4.42, 1.5 , 6.67])}]
French results on model--USE---> lr
[{'mse': 4569.830213673282, 'rmse': 67.60051932990812, 'rho': 0.206

In [32]:
for ngram in range(2,5):  
  for classifier in classifiers:
    results,scores=tfidf(df_fr_train,df_fr_test,classifier,ngram)
    print('French results on model--TF-IDF--->',classifier,' --> n grams:  ',ngram)
    print(results)

French results on model--TF-IDF---> svm  --> n grams:   2
[{'mse': 4.506542709855408, 'rmse': 2.1228619149288557, 'rho': 0.2296193115707354, 'p-value': 1.540246820821582e-13, 'eval_pred': array([5.76164499, 4.29811907, 5.14561943, ..., 5.62058249, 5.25739224,
       4.52374039]), 'eval_labels': array([3.25, 4.08, 1.67, ..., 4.42, 1.5 , 6.67])}]
French results on model--TF-IDF---> knr  --> n grams:   2
[{'mse': 4.999355996035679, 'rmse': 2.2359239691983444, 'rho': 0.03809706147462164, 'p-value': 0.2266309908066629, 'eval_pred': array([5.998, 5.282, 3.734, ..., 6.216, 3.752, 4.214]), 'eval_labels': array([3.25, 4.08, 1.67, ..., 4.42, 1.5 , 6.67])}]
French results on model--TF-IDF---> dt  --> n grams:   2
[{'mse': 4.0927082011892955, 'rmse': 2.0230442904665473, 'rho': 0.3119315513776263, 'p-value': 3.2973333041176752e-24, 'eval_pred': array([3.92, 4.58, 5.5 , ..., 6.58, 3.92, 2.83]), 'eval_labels': array([3.25, 4.08, 1.67, ..., 4.42, 1.5 , 6.67])}]
French results on model--TF-IDF---> lr  

In [33]:
fr_results,fr_scores=use(df_fr_train,df_fr_test,'svm')
fr_results

[{'eval_labels': array([3.25, 4.08, 1.67, ..., 4.42, 1.5 , 6.67]),
  'eval_pred': array([5.36274701, 4.28460411, 3.53859666, ..., 3.86386237, 3.96695503,
         4.16541119]),
  'mse': 3.3313415574707665,
  'p-value': 0.34212419795352644,
  'rho': 0.029937021610930405,
  'rmse': 1.8251963065573977}]

In [34]:
df_fr_test['Score']=fr_scores
del df_fr_test['Sentence_emb']
df_fr_test

,Construction,Sentence,Scores,Score
ID,,,,
fr_0,ingeneral,"J'aime les buissons, et les sapins en général.",3.25,5.362747
fr_1,ingeneral,"J'aime les produits de la mer, et le caviar en...",4.08,4.284604
fr_2,particular,"J'apprécie les conducteurs, et en particulier ...",1.67,3.538597
fr_3,particular,"J'apprécie les politiciens, et en particulier ...",2.42,3.515586
fr_4,ingeneral,"J'aime les huîtres, et le veau en général.",3.25,5.091562
...,...,...,...,...
fr_1004,particular,"J'apprécie les maires, et en particulier les c...",1.50,3.681623
fr_1005,particular,"J'aime les arbres, et en particulier les buiss...",3.83,4.806598
fr_1006,unlike,"Contrairement à le saumon, le poulet est souve...",4.42,3.863862


In [35]:
df_fr_test.to_csv('answer_fr_with_sent.csv') 
files.download('answer_fr_with_sent.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Italian

## Train

In [36]:
data_url = 'https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/train/train_subtask-2/it/'
folds = [pd.read_csv(os.path.join(data_url, 'It-Subtask2-fold_{}.tsv'.format(i)), sep='\t') \
         for i in range(2)] #reading all folds from GitHub

df_it_train = pd.concat(folds[0:2]) 
df_it_train

,ID,Sentence,Score
0,73,"Adoro gli abeti, ma non i cespugli.",4.33
1,106,"Adoro il vitello, ma non i granchi.",3.33
2,203,"Adoro gli animali, un tipo interessante di abete.",1.25
3,466,Amo gli abeti più dei cespugli.,5.18
4,137,"Amo i criceti, un tipo interessante di pappaga...",1.36
...,...,...,...
257,370,"Amo il tacchino, ed anche i granchi.",6.00
258,453,Amo gli alberi più delle betulle.,2.22
259,288,"Amo gli uccelli, ed anche i merli.",3.55
260,386,"Amo i prodotti del mare, ed anche il caviale.",5.08


In [37]:

import base64
import requests
import pandas as pd
df_it_test = pd.read_csv('https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/test/official_test_set_with_labels/subtask-2/It-Subtask2-scores.tsv', sep='\t')
df_it_test=df_it_test.set_index('ID')
scores=[random.uniform(1.0,7.0)]*1009 

df_it_test['Score']=scores
df_it_test 

,Construction,Sentence,Scores,Score
ID,,,,
it_0,ingeneral,"Amo gli arbusti, e le querce in generale.",2.91,5.385794
it_1,particular,"Amo le montagne, e in particolare i centri com...",1.30,5.385794
it_2,comparatives,Amo le foreste più delle case.,5.92,5.385794
it_3,andtoo,"Amo le foreste, ed anche i centri commerciali.",5.60,5.385794
it_4,ingeneral,"Amo i merli, e i gatti in generale.",3.22,5.385794
...,...,...,...,...
it_1004,comparatives,Apprezzo i senatori più degli agricoltori.,5.92,5.385794
it_1005,unlike,"A differenza dei prodotti del mare, il vitello...",4.73,5.385794
it_1006,comparatives,Apprezzo i politici più degli agricoltori.,5.70,5.385794


## Call best model and Save best output Italian

In [38]:
classifiers=['svm','knr','dt','lr']
max_rho=(float('-inf'),"")

In [ ]:
for classifier in classifiers:
  results,scores=use(df_it_train,df_it_test,classifier)
  print('Italian results on model--USE--->',classifier)
  print(results)

In [ ]:
for ngram in range(2,5):  
  for classifier in classifiers:
    results,scores=tfidf(df_it_train,df_it_test,classifier,ngram)
    print('Italian results on model--TF-IDF--->',classifier,' --> n grams:  ',ngram)
    print(results)

In [ ]:
it_results,it_scores=use(df_it_train,df_it_test,'svm')
it_results

In [ ]:
df_it_test['Score']=it_scores
del df_it_test['Sentence_emb']
df_it_test

In [ ]:
df_it_test.to_csv('answer_it_with_sent.csv') 
files.download('answer_it_with_sent.csv')

# Concat All best_outputs for 3 langs

In [ ]:
df_en_test

In [ ]:
del df_en_test['Sentence']
df_en_test

In [ ]:
df_en_test.to_csv('answer_en.csv') 
files.download('answer_en.csv')

In [ ]:
df_fr_test

In [ ]:
del df_fr_test['Sentence']
df_fr_test

In [ ]:
df_fr_test.to_csv('answer_fr.csv') 
files.download('answer_fr.csv')

In [ ]:
df_it_test

In [ ]:
del df_it_test['Sentence']
df_it_test

In [ ]:
df_it_test.to_csv('answer_it.csv') 
files.download('answer_it.csv')